<a href="https://colab.research.google.com/github/rkp74/Fake-News-Classifier/blob/main/Fake_News_Classifier_using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
import pandas as pd

In [57]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [58]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Fake_news_dataset/train.csv')

In [59]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [60]:
df = df.dropna()

In [61]:
X = df.drop('label',axis=1)

In [62]:
y = df['label']

In [63]:
X.shape

(18285, 4)

In [64]:
y.shape

(18285,)

In [65]:
import tensorflow as tf

In [66]:
tf.__version__

'2.11.0'

In [67]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [68]:
voc_size = 5000

In [69]:
messages = X.copy()

In [70]:
messages.reset_index(inplace = True)

In [71]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [72]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()
     

In [73]:
## Datset PreProcessing
corpus = []
for i in range(0,len(messages)):
  print(i)
  review = re.sub('[^a-zA-Z]', ' ' ,messages['title'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [74]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [75]:
one_hot_repr = [one_hot(words,voc_size) for words in corpus]
one_hot_repr

[[4565, 2162, 2589, 4133, 388, 149, 3604, 1758, 1188, 2506],
 [2567, 2577, 2659, 1309, 2810, 3139, 1915],
 [3743, 4235, 2381, 2202],
 [2092, 818, 2024, 1178, 2911, 1206],
 [4283, 2810, 2763, 2267, 4112, 3334, 2810, 2460, 294, 4693],
 [4163,
  151,
  2926,
  6,
  303,
  1968,
  3837,
  4461,
  2598,
  3609,
  1788,
  254,
  656,
  1760,
  1915],
 [4575, 2199, 278, 2962, 3672, 1803, 4759, 1122, 2729, 2700, 3230],
 [3315, 3712, 4472, 4401, 1643, 1999, 1968, 1627, 2729, 2700, 3230],
 [4079, 705, 3532, 1901, 2018, 1640, 773, 803, 1968, 1542],
 [1712, 2122, 1962, 1798, 683, 1161, 3779, 663],
 [1249, 154, 435, 244, 1871, 1814, 3322, 4135, 4170, 3156, 1398],
 [1178, 2994, 388, 1640, 1968, 1643],
 [4492, 4449, 3123, 3755, 2760, 1371, 3389, 2320, 2798],
 [3905, 2158, 2046, 1505, 4653, 1827, 4190, 2729, 2700, 3230],
 [3871, 2828, 4312, 2513, 4222, 2729, 2700, 3230],
 [4374, 4723, 805, 2877, 602, 3988, 760, 4161, 957, 1094],
 [4432, 840, 2577],
 [4900, 4361, 741, 1994, 1968, 2084, 4897, 1915],
 [4

In [76]:
sent_length = 20
embedded_docs = pad_sequences(one_hot_repr,padding = 'pre' , maxlen = sent_length)
print(embedded_docs)

[[   0    0    0 ... 1758 1188 2506]
 [   0    0    0 ... 2810 3139 1915]
 [   0    0    0 ... 4235 2381 2202]
 ...
 [   0    0    0 ... 2729 2700 3230]
 [   0    0    0 ... 4342 1027 2390]
 [   0    0    0 ... 1326 1333  939]]


In [77]:
len(embedded_docs)

18285

In [78]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4565,
       2162, 2589, 4133,  388,  149, 3604, 1758, 1188, 2506], dtype=int32)

In [79]:
model = Sequential()
model.add(Embedding(voc_size,40,input_length = sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(loss='binary_crossentropy' , optimizer ='adam' , metrics =['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [80]:
len(embedded_docs) , y.shape

(18285, (18285,))

In [81]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [82]:
X_final.shape , y_final.shape

((18285, 20), (18285,))

In [83]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X_final , y_final, test_size = 0.33 , random_state = 42) 

In [84]:
X_train.shape

(12250, 20)

In [85]:
y_train.shape

(12250,)

# Model Training

In [86]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs = 20,batch_size = 32)

Epoch 1/20
383/383 [==============================] - 16s 34ms/step - loss: 0.2713 - accuracy: 0.8768 - val_loss: 0.1857 - val_accuracy: 0.9220
Epoch 2/20
383/383 [==============================] - 4s 11ms/step - loss: 0.1323 - accuracy: 0.9481 - val_loss: 0.1980 - val_accuracy: 0.9176
Epoch 3/20
383/383 [==============================] - 3s 7ms/step - loss: 0.0894 - accuracy: 0.9678 - val_loss: 0.2345 - val_accuracy: 0.9148
Epoch 4/20
383/383 [==============================] - 3s 8ms/step - loss: 0.0595 - accuracy: 0.9806 - val_loss: 0.2626 - val_accuracy: 0.9143
Epoch 5/20
383/383 [==============================] - 5s 12ms/step - loss: 0.0364 - accuracy: 0.9879 - val_loss: 0.3580 - val_accuracy: 0.9158
Epoch 6/20
383/383 [==============================] - 3s 9ms/step - loss: 0.0230 - accuracy: 0.9933 - val_loss: 0.3746 - val_accuracy: 0.9152
Epoch 7/20
383/383 [==============================] - 3s 7ms/step - loss: 0.0153 - accuracy: 0.9953 - val_loss: 0.4691 - val_accuracy: 0.9122
Ep

In [87]:
y_pred = np.argmax(model.predict(X_test), axis=1)

189/189 [==============================] - 1s 2ms/step


In [88]:
from sklearn.metrics import confusion_matrix

In [91]:
y_test.shape , y_pred.shape

((6035,), (6035,))

In [92]:
confusion_matrix(y_test,y_pred)

array([[3419,    0],
       [2616,    0]])

In [90]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.5665285832642917